Make your own Machine Learning Tutorial ipython notebook page outlining an implementation of Machine Learning.

It could investigate setting up  a ML technology stack:

1.  Azure  azure.microsoft.com/Services/MachineLearning

2. H20 ML  - https://www.h2o.ai/

3. Tensor Flow


Implementing a demonstration page in Ipython for at Deep Learning Tutorial:

1. http://deeplearning.net/tutorial/

2. https://www.datacamp.com/community/tutorials/deep-learning-python

3. https://pythonspot.com/en/machine-learning/

Deep learning refers to neural networks with multiple hidden layers that can learn increasingly abstract representations of the input data. Keras is a simple,modular library used for deep learning in Python, especially for beginners. <br/>
I used keras to implement neural network with tensorflow machine learning library as backend.

In [25]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam

##### About Data
Wisconsin Breast Cancer dataset has 699 observations collected using fine-needle tissue from a mass under skin. It has 11 variables of which nine are predictor variables (cytological characteristics used to identify mass as benign or malign), ID and a class variable (has values 2 for benign, 4 for malignant). 458 of the samples are benign and 241 are malignant. <br/>

There are 16 samples with missing data. Data file doesn't have column names and they are listed in a separate file. <br/>
'class' variable is the outcome (2=benign, 4=malignant).

In [26]:
data = pd.read_csv('data/breast-cancer-wisconsin.data.txt', sep=",", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


###### Adding column headers to data

In [27]:
data.columns = ["ID", "clumpThickness", "sizeUniformity",
"shapeUniformity", "maginalAdhesion",
"singleEpithelialCellSize", "bareNuclei",
"blandChromatin", "normalNucleoli", "mitosis", "class"]
data.head(10)

,ID,clumpThickness,sizeUniformity,shapeUniformity,maginalAdhesion,singleEpithelialCellSize,bareNuclei,blandChromatin,normalNucleoli,mitosis,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [28]:
data.shape

(699, 11)

###### Counting the different classes of 'class' variable

In [29]:
data['class'].value_counts()

2    458
4    241
Name: class, dtype: int64

###### Converting classes '2','4' to binary '0'(benign) and '1'(malignant)

In [30]:
#data['class'].replace(2, 0, inplace=True)
#data['class'].replace(4, 1, inplace=True)
#data['class'].value_counts()

In [31]:
data.dtypes

ID                           int64
clumpThickness               int64
sizeUniformity               int64
shapeUniformity              int64
maginalAdhesion              int64
singleEpithelialCellSize     int64
bareNuclei                  object
blandChromatin               int64
normalNucleoli               int64
mitosis                      int64
class                        int64
dtype: object

######  Converting 'class' variable from int to string

In [32]:
data['class'] = data['class'].astype(str)

In [33]:
#Converting 'object' to numeric data type
data['bareNuclei'] = pd.to_numeric(data['bareNuclei'], errors='coerce')
#data['bareNuclei'] = data['bareNuclei'].astype(str).astype(int)

In [34]:
data.dtypes

ID                            int64
clumpThickness                int64
sizeUniformity                int64
shapeUniformity               int64
maginalAdhesion               int64
singleEpithelialCellSize      int64
bareNuclei                  float64
blandChromatin                int64
normalNucleoli                int64
mitosis                       int64
class                        object
dtype: object

In [35]:
data.isnull().sum()

ID                           0
clumpThickness               0
sizeUniformity               0
shapeUniformity              0
maginalAdhesion              0
singleEpithelialCellSize     0
bareNuclei                  16
blandChromatin               0
normalNucleoli               0
mitosis                      0
class                        0
dtype: int64

###### Replacing missing values with mean value of column

In [36]:
data["bareNuclei"].fillna(data["bareNuclei"].mean(), inplace=True)

In [37]:
#Now check for missing values in dataframe
data.isnull().sum()

ID                          0
clumpThickness              0
sizeUniformity              0
shapeUniformity             0
maginalAdhesion             0
singleEpithelialCellSize    0
bareNuclei                  0
blandChromatin              0
normalNucleoli              0
mitosis                     0
class                       0
dtype: int64

In [38]:
data = data.drop(['ID'], axis=1)
data.columns

Index(['clumpThickness', 'sizeUniformity', 'shapeUniformity',
       'maginalAdhesion', 'singleEpithelialCellSize', 'bareNuclei',
       'blandChromatin', 'normalNucleoli', 'mitosis', 'class'],
      dtype='object')

In [39]:
data.describe()

,clumpThickness,sizeUniformity,shapeUniformity,maginalAdhesion,singleEpithelialCellSize,bareNuclei,blandChromatin,normalNucleoli,mitosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.601852,2.438364,3.053634,1.715078
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [40]:
data['mitosis'].value_counts()

1     579
2      35
3      33
10     14
4      12
7       9
8       8
5       6
6       3
Name: mitosis, dtype: int64

In [41]:
#np.ravel(data['class'])

###### Splitting data into input and output variables

In [42]:
X = data.iloc[:,0:9].values
y = data.iloc[:,9].values

##### Encoding Output Variable
Encoding is to reshape output variable from a vector to a matrix with a boolean for each class value and whether or not a given instance has that class value or not.<br/>

This is done here by encoding 'class' strings to integers using LabelEncoder() from scikit-learn. 

In [45]:
#encoding class values as integers
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values

###### Splitting data into train and test variables

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=0) 

In [47]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(489, 9)
(489, 2)
(210, 9)
(210, 2)


###### Neural Networks
We start building neural network model using Sequential(). Keras deals with stack of layers and we add each layer using add(). 

This function takes input shape, activation function as parameters. The activation function can take values 'sigmoid', 'tanh' and 'relu'. Recent papers on neural networks suggested that 'relu' activation functio performs better than the other 2 in most cases. So I used relu as activation here. <br/>

Since my data has 9 features, I took input_shape as 9. <br/>

In first layer I used 12 neurons and in the next layer(hidden layer) I have taken 8 neurons with same activation function. In the output layer I have 2 neurons and used softmax activation which gives probabilities.

In [48]:
model = Sequential()

model.add(Dense(12, input_shape=(9,), activation='relu'))
model.add(Dense(8, activation='relu'))
#model.add(Dense(6, activation='relu'))
model.add(Dense(2, activation='softmax'))

##### Compilation
Before training model on our data, we have to do compilation using compile() of Tensorflow library. It has optimizer, loss function, metrics as arguments.

optimizer is used to browse through weights of netwrok, loss function is to minimize the model on the objective specified and metrics are the metrics we want to analyze the performance of our model.<br/>

I used adam algorithm and crossentropy as optimizer and loss function respectively. As it is a classification , I have givne accuracy as metrics.

In [49]:
model.compile(optimizer=Adam(lr=0.04), loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                120       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 18        
Total params: 242.0
Trainable params: 242
Non-trainable params: 0.0
_________________________________________________________________


In [51]:
model.fit(X_train, y_train, epochs=350)

Epoch 1/350
489/489 [==============================] - 1s - loss: 1.1749 - acc: 0.5767     
Epoch 2/350
489/489 [==============================] - 0s - loss: 0.3314 - acc: 0.9202     
Epoch 3/350
489/489 [==============================] - 0s - loss: 0.1902 - acc: 0.9530     
Epoch 4/350
489/489 [==============================] - 0s - loss: 0.1363 - acc: 0.9652     
Epoch 5/350
489/489 [==============================] - 0s - loss: 0.1098 - acc: 0.9673     
Epoch 6/350
489/489 [==============================] - 0s - loss: 0.0936 - acc: 0.9755     
Epoch 7/350
489/489 [==============================] - 0s - loss: 0.1020 - acc: 0.9734     
Epoch 8/350
489/489 [==============================] - 0s - loss: 0.0814 - acc: 0.9755     
Epoch 9/350
489/489 [==============================] - 0s - loss: 0.0755 - acc: 0.9693     
Epoch 10/350
489/489 [==============================] - 0s - loss: 0.0732 - acc: 0.9734     
Epoch 11/350
489/489 [==============================] - 0s - loss: 0.0686 - acc

489/489 [==============================] - 0s - loss: 0.0427 - acc: 0.9877     
Epoch 90/350
489/489 [==============================] - 0s - loss: 0.0310 - acc: 0.9877     
Epoch 91/350
489/489 [==============================] - 0s - loss: 0.0482 - acc: 0.9796     
Epoch 92/350
489/489 [==============================] - 0s - loss: 0.0594 - acc: 0.9836     
Epoch 93/350
489/489 [==============================] - 0s - loss: 0.0657 - acc: 0.9755     
Epoch 94/350
489/489 [==============================] - 0s - loss: 0.0511 - acc: 0.9836     
Epoch 95/350
489/489 [==============================] - 0s - loss: 0.0493 - acc: 0.9857     
Epoch 96/350
489/489 [==============================] - 0s - loss: 0.0578 - acc: 0.9816     
Epoch 97/350
489/489 [==============================] - 0s - loss: 0.0534 - acc: 0.9796     
Epoch 98/350
489/489 [==============================] - 0s - loss: 0.0796 - acc: 0.9673     
Epoch 99/350
489/489 [==============================] - 0s - loss: 0.0672 - acc: 0.

489/489 [==============================] - 0s - loss: 0.0240 - acc: 0.9918     
Epoch 177/350
489/489 [==============================] - 0s - loss: 0.0244 - acc: 0.9898     
Epoch 178/350
489/489 [==============================] - 0s - loss: 0.0193 - acc: 0.9918     
Epoch 179/350
489/489 [==============================] - 0s - loss: 0.0221 - acc: 0.9918     
Epoch 180/350
489/489 [==============================] - 0s - loss: 0.0215 - acc: 0.9898        
Epoch 181/350
489/489 [==============================] - 0s - loss: 0.0218 - acc: 0.9918     
Epoch 182/350
489/489 [==============================] - 0s - loss: 0.0210 - acc: 0.9918        
Epoch 183/350
489/489 [==============================] - 0s - loss: 0.0257 - acc: 0.9877     
Epoch 184/350
489/489 [==============================] - 0s - loss: 0.0384 - acc: 0.9898     
Epoch 185/350
489/489 [==============================] - 0s - loss: 0.0418 - acc: 0.9775     
Epoch 186/350
489/489 [==============================] - 0s - loss: 

489/489 [==============================] - 0s - loss: 0.0335 - acc: 0.9857     
Epoch 264/350
489/489 [==============================] - 0s - loss: 0.0499 - acc: 0.9857     
Epoch 265/350
489/489 [==============================] - 0s - loss: 0.0341 - acc: 0.9857     
Epoch 266/350
489/489 [==============================] - 0s - loss: 0.0453 - acc: 0.9877     
Epoch 267/350
489/489 [==============================] - 0s - loss: 0.0521 - acc: 0.9775     
Epoch 268/350
489/489 [==============================] - 0s - loss: 0.0420 - acc: 0.9816     
Epoch 269/350
489/489 [==============================] - 0s - loss: 0.0405 - acc: 0.9857     
Epoch 270/350
489/489 [==============================] - 0s - loss: 0.0393 - acc: 0.9836     
Epoch 271/350
489/489 [==============================] - 0s - loss: 0.0396 - acc: 0.9816     
Epoch 272/350
489/489 [==============================] - 0s - loss: 0.0215 - acc: 0.9918     
Epoch 273/350
489/489 [==============================] - 0s - loss: 0.0359

489/489 [==============================] - 0s - loss: 0.0193 - acc: 0.9918     


##### Evaluate and Predict Model
After training the neural network model on train data, I evaluated the model on my test data. This gives scores for every sample (input,output) and metrics(accuracy in this case).<br/>

I got an accuracy of 95% on the test data.

In [55]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/210 [===>..........................] - ETA: 0s
acc: 94.76%


In [56]:
y_pred = model.predict(X_test)

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)

In [57]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test_class, y_pred_class))
print(confusion_matrix(y_test_class, y_pred_class))

             precision    recall  f1-score   support

          0       0.97      0.95      0.96       135
          1       0.91      0.95      0.93        75

avg / total       0.95      0.95      0.95       210

[[128   7]
 [  4  71]]
